In [ ]:
%matplotlib inline
import os
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyemu

In [ ]:
import freyberg_setup as fs
pst_name = fs.PST_NAME_ZN
working_dir = fs.WORKING_DIR_ZN
fs.setup_pest_zn()

## Given a zone array let's plot it up

In [ ]:
zones = np.loadtxt(os.path.join(working_dir, 'hk.zones'))
plt.imshow(zones)
plt.colorbar()

## run pestpp

In [ ]:
pst = pyemu.Pst(os.path.join(working_dir,pst_name))
pst.control_data.noptmax = 10
pst.write(os.path.join(working_dir,pst_name))
pyemu.helpers.run("pestpp {0}".format(pst_name),cwd=working_dir)

In [ ]:
pst.phi

In [ ]:
pst.phi_components

In [ ]:
pst.res.loc[pst.nnz_obs_names,:]

In [ ]:
ax = plt.subplot(111,aspect="equal") 
wl_names = [n for n in pst.nnz_obs_names if n.startswith("c")]
ax.scatter(pst.res.loc[wl_names,"modelled"],pst.res.loc[wl_names,"measured"],marker='.')
ylim,xlim = ax.get_ylim(),ax.get_xlim()
mn = min(ylim[0],xlim[0])
mx = max(ylim[1],xlim[1])
ax.plot([mn,mx],[mn,mx])
ax.set_ylim(mn,mx)
ax.set_xlim(mn,mx)

In [ ]:
df_paru = pd.read_csv(os.path.join(working_dir,pst_name.replace(".pst",".par.usum.csv")),index_col=0)
df_paru

In [ ]:
axes = pyemu.helpers.plot_summary_distributions(df_paru,subplots=True)

Why is ``rch_1`` unaffected by calibration?

In [ ]:
df_predu = pd.read_csv(os.path.join(working_dir,pst_name.replace(".pst",".pred.usum.csv")),index_col=0)

In [ ]:
figs, axes = pyemu.helpers.plot_summary_distributions(df_predu,subplots=True)
for ax in axes:
    fname = ax.get_title().lower()
    ylim = ax.get_ylim()
    v = pst.observation_data.loc[fname,"obsval"]
    ax.plot([v,v],ylim,"k--")
    ax.set_ylim(ylim)

## Now we have some variability in the forecast outcomes...why?

## As far as the "truth" is concerned, we are doing better for most forecasts.  But we are especially bad for the tranvel time forecast - why?  What affects travel time?

## Zone Burner approach

In [ ]:
np.unique(zones)

In [ ]:
a = zones.copy()
newk = [100,222,303,444,515,676]

In [ ]:
for i in range(len(newk)):
    print(i)
    a[zones==i+1] = newk[i]

In [ ]:
a